In [1]:
import itertools
from scipy.optimize import linear_sum_assignment
import numpy as np
from tqdm import tqdm
from collections import Counter

In [2]:
import os 
os.environ['CPLEX_STUDIO_DIR201'] = "/opt/ibm/ILOG/CPLEX_Studio_Community201"
os.environ['CPLEX_STUDIO_KEY'] = "xxxx"
from docplex.mp.model import Model
from docplex.mp.environment import Environment

In [3]:
permutations = [''.join(x) for x in itertools.permutations(['1','2','3','4','5','6','7'], 7)]
permutations1 = ['12' + ''.join(x) for x in itertools.permutations(['3','4','5','6','7'], 5)]

In [4]:
N = len(permutations)
N

5040

In [5]:
MAX_COST = 7

In [6]:
p2i = {}
for i,p in enumerate(permutations):
    p2i[p] = i

In [7]:
len(permutations)

5040

In [8]:
permutations = np.array(permutations)
permutations_set = set(permutations)
#np.random.shuffle(permutations)

In [9]:
def finalize(strings):
    replace_dict = {
        "1": '🎅', 
        "2": '🤶', 
        "3": '🦌', 
        "4": '🧝', 
        "5": '🎄', 
        "6": '🎁', 
        "7": '🎀', 
    }
    ans = strings.copy()
    for i in range(3):
        for k,v in replace_dict.items():
            ans[i] = ans[i].replace(k, v)
    return ans

In [10]:
def get_cost(p1, p2):
    if p1 == p2:
        return 1000
    for i in range(1, 8):
        if p1[i:] == p2[:-i]:
            if i == 7:
                return 7 - 1e-2
            if i > 1 and p1[:i] == p2[-i:]:
                return 10000
            s = p1[:i] + p2
            s = s[1:-1]
            for k in range(len(s) - 7):
                if s[k:k+7] in permutations_set:
                    return 10000
            return i #- (2**i - 2) * 2**(-13)
    return 10000

p1, p2 = '1234567', '3456721'
get_cost(p1, p2)

def get_dmat(permutations):
    L = len(permutations)
    var_idx = []
    cost = {}
    for i,p1 in enumerate(tqdm(permutations)):
        for j,p2 in enumerate(permutations):
            cost_ij = get_cost(p1, p2)
            if cost_ij <= MAX_COST:
                var_idx.append((i, j))
                cost[(i,j)] = cost_ij
    return var_idx, cost

var_idx, all_cost = get_dmat(permutations)

len(var_idx)

100%|███████████████████████████████████████| 5040/5040 [01:16<00:00, 66.18it/s]


24212160

In [11]:
def get_pair_idx(var_idx):
    firsts = {}
    lasts = {}
    for i,j in var_idx:
        if not firsts.get(j, False):
            firsts[j] = [i]
        else:
            firsts[j].append(i)
        if not lasts.get(i, False):
            lasts[i] = [j]
        else:
            lasts[i].append(j)
    return firsts, lasts


firsts, lasts = get_pair_idx(var_idx)

In [12]:
len(firsts[0]), len(lasts[0])

(4804, 4804)

In [13]:
env = Environment()
env.print_information()

* system is: Linux 64bit
* Python version 3.8.12, located at: /home/jpuget/miniconda3/envs/cplex201/bin/python
* docplex is present, version is 2.18.200
* CPLEX library is present, version is 20.1.0.0, located at: /home/jpuget/miniconda3/envs/cplex201/lib/python3.8/site-packages
* pandas is present, version is 1.3.4


In [14]:
mdl = Model("santa")
next_perm = mdl.integer_var_dict(var_idx)
visits = mdl.integer_var_list(range(N))
mdl.print_information()

Model: santa
 - number of variables: 24217200
   - binary=0, integer=24217200, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [15]:
mdl.add_constraints(
    visits[i] >= 3
    for i in range(120))

mdl.add_constraints(
    visits[i] >= 1
    for i in range(120, N))

mdl.print_information()

Model: santa
 - number of variables: 24217200
   - binary=0, integer=24217200, continuous=0
 - number of constraints: 5040
   - linear=5040
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [16]:
mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i in firsts[j]) == visits[j]
    for j in range(N))

mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for j in lasts[i]) == visits[i]
    for i in range(N))

mdl.print_information()

Model: santa
 - number of variables: 24217200
   - binary=0, integer=24217200, continuous=0
 - number of constraints: 15120
   - linear=15120
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [17]:
def rot(s, i=1):
    if i <= 0:
        return s
    return s[i:] + s[:i]

def get_canon(s):
    best = s
    for i in range(len(s)-1):
        s = rot(s)
        if s[0] < best[0]:
            best = s
    return best

def get_1_cycle(s):
    s = get_canon(s)
    cycle = [rot(s, i) for i in range(len(s))]
    cycle = [p2i[p] for p in cycle]
    cycle = cycle + [cycle[0]]
    return cycle
    
cycles1 = []
for p in permutations:
    if p != get_canon(p):
        continue
    cycles1.append(get_1_cycle(p))

In [18]:
len([(i,j) for cycle in cycles1 for i,j in zip(cycle[:-1],cycle[1:])])


5040

In [19]:
def get_inner(cycle):
    inner = []
    for i,j in zip(cycle[:-1],cycle[1:]):
        inner.extend([(k, j) for k in firsts[j] if k not in cycle])
    return inner

def get_outer(cycle):
    outer = []
    for i,j in zip(cycle[:-1],cycle[1:]):
        outer.extend([(i, k) for k in lasts[i] if k not in cycle])
    return outer


In [20]:
mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i,j in get_inner(cycle)) >= 1
    for cycle in cycles1)

mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i,j in get_outer(cycle)) >= 1
    for cycle in cycles1)

mdl.print_information()

Model: santa
 - number of variables: 24217200
   - binary=0, integer=24217200, continuous=0
 - number of constraints: 16560
   - linear=16560
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [21]:
def remove_start(s, start):
    idx = s.index(start)
    s = s[idx:] + s[:idx]
    return s[0] + get_canon(s[1:])

def get_2_cycle(p, start):
    p = remove_start(p, start)
    p1 = p[1:]
    inner = [rot(p1, i) for i in range(len(p1))]
    cycle = [rot(q+start, j) for q in inner for j in range(len(p))]
    #cycle1 = [get_1_cycle(q+start) for q in inner]
    #cycle = get_canon(cycle)
    cycle = [p2i[p] for p in cycle]
    cycle = cycle + [cycle[0]]
    return p, cycle

cycles2 = {}

for p in tqdm(permutations):
    for start in p:
        p1 = remove_start(p, start)
        if cycles2.get(p1, None) is not None:
            continue
        p1, cycle = get_2_cycle(p, start)
        cycles2[p1] = cycle

cycles2['2134567']

len(cycles2)

100%|████████████████████████████████████| 5040/5040 [00:00<00:00, 34261.80it/s]


840

In [22]:
cycles2['1234567']

[873,
 1744,
 2610,
 3456,
 4200,
 4320,
 0,
 1745,
 2612,
 3462,
 4224,
 4440,
 720,
 153,
 2613,
 3464,
 4230,
 4464,
 840,
 1473,
 304,
 3465,
 4232,
 4470,
 864,
 1689,
 2224,
 450,
 4233,
 4472,
 870,
 1731,
 2536,
 2970,
 576,
 4473,
 872,
 1741,
 2596,
 3378,
 3696,
 600,
 873]

In [23]:
len(get_inner(cycles2['1234567']))

200568

In [24]:
mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i,j in get_inner(cycle)) >= 1
    for cycle in cycles2.values())

mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i,j in get_outer(cycle)) >= 1
    for cycle in cycles2.values())

mdl.print_information()

Model: santa
 - number of variables: 24217200
   - binary=0, integer=24217200, continuous=0
 - number of constraints: 18240
   - linear=18240
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [25]:
objective = mdl.sum(next_perm[idx] * all_cost[idx]
                    for idx in var_idx)
                          
# Set objective function
mdl.minimize(objective)

mdl.print_information()

mdl.parameters.mip.tolerances.mipgap = 0.00
mdl.parameters.timelimit = 50000 
mdl.parameters.threads = 5
mdl.parameters.mip.cuts.gomory = -1

mdl.solve(log_output=True)
mdl.report()
   

Model: santa
 - number of variables: 24217200
   - binary=0, integer=24217200, continuous=0
 - number of constraints: 18240
   - linear=18240
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
Checking license ...
License found. [1.21 s]
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 5
CPXPARAM_RandomSeed                              202001241
CPXPARAM_MIP_Cuts_Gomory                         -1
CPXPARAM_TimeLimit                               50000
CPXPARAM_MIP_Tolerances_MIPGap                   0
Presolve has eliminated 5040 rows and 0 columns...
Presolve has eliminated 5040 rows and 0 columns...
Tried aggregator 1 time.
Presolve has eliminated 5040 rows and 0 columns...
MIP Presolve eliminated 5040 rows and 0 columns.
Reduced MIP has 13200 rows, 24217200 columns, and 433802880 nonzeros.
Reduced MIP has 0 binaries, 24217200 generals, 0 SOSs, and 0 indicators

In [26]:
sol = mdl.solution
sol

s = sol.get_value_dict(next_perm)
s

s_cost = {}
s_cost1 = {}
s_sol = {}
for k,v in s.items():
    if v > 0:
        s_cost[k] = all_cost[k] * v
        s_cost1[k] = all_cost[k] 
        s_sol[k] = v

s_cost

C = Counter(s_cost.values())
C1 = Counter(s_cost1.values())
C, C1

(Counter({6.99: 240, 1.0: 4440, 2.0: 600}),
 Counter({6.99: 240, 1: 4440, 2: 600}))

In [28]:
np.sum([np.round(k) * v for k,v in C.items()])

7320.0